In [1]:
import os
os.chdir(r'C:\Users\Veena Nigam\Desktop\Python project\Data Cleaning Projects')
os.getcwd()

'C:\\Users\\Veena Nigam\\Desktop\\Python project\\Data Cleaning Projects'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing the numpy and pandas package
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 


# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
df = pd.read_csv("Restaurant_tips_Missing.csv")

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         239 non-null    float64
 2   sex         242 non-null    object 
 3   smoker      244 non-null    object 
 4   day         239 non-null    object 
 5   time        241 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [7]:
# counting missing values
df.apply(lambda x: sum(x.isnull()))

total_bill    0
tip           5
sex           2
smoker        0
day           5
time          3
size          0
dtype: int64

In [8]:
#method 2:
#calculatin no. of missing values for each column and it's percentage
def percentage_of_miss():
  df1=df[df.columns[df.isnull().sum()>=1]]
  total_miss = df1.isnull().sum().sort_values(ascending=False)
  percent_miss = (df1.isnull().sum()/df1.isnull().count()).sort_values(ascending=False)
  missing_data = pd.concat([total_miss, percent_miss], axis=1, keys=['Number of Missing', 'Percentage'])
  return(missing_data)

In [9]:
percentage_of_miss()

,Number of Missing,Percentage
day,5,0.020492
tip,5,0.020492
time,3,0.012295
sex,2,0.008197


#### Four Columns- day,tip,time,sex are having missing values. Where tip is the only numerical variable among them, others are categorical variables.

### Handling Missing Values

### Tip Column

#### Replacing missing values with Mean

In [10]:
# Determine the tip values:
tip = df.pivot_table(values='tip', index = 'size')
# Get a Boolean variable specifying missing tip values
miss_bool = df['tip'].isnull()
# Impute data and check missing values before and after imputation to confirm
print('Original #missing: %d' % sum(miss_bool))

df.loc[miss_bool,'tip'] = df.loc[miss_bool,'size'].apply(lambda x: tip.at[x,'tip'])
print('Final tip values # missing: %d'%sum(df['tip'].isnull()))

Original #missing: 5
Final tip values # missing: 0


#### Day Column

#### Replacing missing values with mode

In [11]:
from scipy.stats import mode

day_mode = df.pivot_table(values='day',columns='sex',aggfunc=(lambda x:mode(x).mode[0]))
print('Mode for each sex Column:')
print(day_mode)

miss_bool =  df['day'].isnull()

print('\nOriginal #missing: %d' %sum(miss_bool))
df.loc[miss_bool,'day']= df.loc[miss_bool,'sex'].apply(lambda x:day_mode[x])
print(sum(df['day'].isnull()))

Mode for each sex Column:
sex Female Male
day   Thur  Sat

Original #missing: 5
0


#### Time Column

In [29]:
from scipy.stats import mode

time_mode = df.pivot_table(values='time',columns='day',aggfunc=(lambda x:mode(x).mode[0]))
print('Mode for each day Column:')
print(time_mode)

miss_bool =  df['time'].isnull()

print('\nOriginal #missing: %d' %sum(miss_bool))
df.loc[miss_bool,'time']= df.loc[miss_bool,'day'].apply(lambda x:time_mode[x])
print(sum(df['time'].isnull()))

Mode for each day Column:
day      Fri     Sat     Sun   Thur
time  Dinner  Dinner  Dinner  Lunch

Original #missing: 0
0


#### Sex Column

In [25]:
from scipy.stats import mode

sex_mode = df.pivot_table(values='sex', columns= 'time',aggfunc=(lambda x:mode(x).mode[0]))
print('Mode for each day and time Column:')
print(sex_mode)

miss_bool =  df['sex'].isnull()

print('\nOriginal #missing: %d' %sum(miss_bool))
df.loc[miss_bool,'sex']= df.loc[miss_bool,'time'].apply(lambda x:sex_mode[x])
print(sum(df['sex'].isnull()))

Mode for each day and time Column:
time Dinner   Lunch
sex    Male  Female

Original #missing: 2
0


In [28]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

#### All missing values are handled either by mean or mode